In [1]:
# DATA CLEANING/TRANSFORMATION
# Import firstnotebook:
%run 1notebook.ipynb

# Import dependencies:
import numpy as np
import csv
import os
from datetime import datetime, timedelta

{'meta': {'code': 200},
 'response': {'holidays': [{'country': {'id': 'ca', 'name': 'Canada'},
                            'date': {'datetime': {'day': 1,
                                                  'month': 1,
                                                  'year': 2010},
                                     'iso': '2010-01-01'},
                            'description': "New Year's Day is the first day of "
                                           'a new year according to the '
                                           'Gregorian calendar used in Canada '
                                           'and many other countries.',
                            'locations': 'All',
                            'name': "New Year's Day",
                            'states': 'All',
                            'type': ['National holiday']},
                           {'country': {'id': 'ca', 'name': 'Canada'},
                            'date': {'datetime': {'day': 2,
               

                            'type': ['Local holiday']},
                           {'country': {'id': 'ca', 'name': 'Canada'},
                            'date': {'datetime': {'day': 30,
                                                  'month': 5,
                                                  'year': 2010},
                                     'iso': '2010-05-30'},
                            'description': 'Trinity Sunday is the first Sunday '
                                           'after Pentecost and is celebrated '
                                           'by many Christians in Canada.',
                            'locations': 'All',
                            'name': 'Trinity Sunday',
                            'states': 'All',
                            'type': ['Christian']},
                           {'country': {'id': 'ca', 'name': 'Canada'},
                            'date': {'datetime': {'day': 3,
                                                  'month': 

                            'type': ['Observance', 'Christian']},
                           {'country': {'id': 'ca', 'name': 'Canada'},
                            'date': {'datetime': {'day': 5,
                                                  'month': 11,
                                                  'year': 2010},
                                     'iso': '2010-11-05'},
                            'description': 'Diwali, also known as the Festival '
                                           'of Lights, marks the beginning of '
                                           'the Hindu New Year.',
                            'locations': 'All',
                            'name': 'Diwali/Deepavali',
                            'states': 'All',
                            'type': ['Observance']},
                           {'country': {'id': 'ca', 'name': 'Canada'},
                            'date': {'datetime': {'day': 7,
                                                  'hour'

In [2]:
# Generate descriptive statistics:
walmart_data.describe()

Store           Dept   Weekly_Sales    Temperature  \
count  421570.000000  421570.000000  421570.000000  421570.000000   
mean       22.200546      44.260317   15981.258123      60.090059   
std        12.785297      30.492054   22711.183519      18.447931   
min         1.000000       1.000000   -4988.940000      -2.060000   
25%        11.000000      18.000000    2079.650000      46.680000   
50%        22.000000      37.000000    7612.030000      62.090000   
75%        33.000000      74.000000   20205.852500      74.280000   
max        45.000000      99.000000  693099.360000     100.140000   

          Fuel_Price      MarkDown1      MarkDown2      MarkDown3  \
count  421570.000000  150681.000000  111248.000000  137091.000000   
mean        3.361027    7246.420196    3334.628621    1439.421384   
std         0.458515    8291.221345    9475.357325    9623.078290   
min         2.472000       0.270000    -265.760000     -29.100000   
25%         2.933000    2240.270000      41.600000       5.080000   
50%         3.452000    5347.450000     192.000000      24.600000   
75%         3.738000    9210.900000    1926.940000     103.990000   
max         4.468000   88646.760000  104519.540000  141630.610000   

           MarkDown4      MarkDown5            CPI   Unemployment  \
count  134967.000000  151432.000000  421570.000000  421570.000000   
mean     3383.168256    4628.975079     171.201947       7.960289   
std      6292.384031    5962.887455      39.159276       1.863296   
min         0.220000     135.160000     126.064000       3.879000   
25%       504.220000    1878.440000     132.022667       6.891000   
50%      1481.310000    3359.450000     182.318780       7.866000   
75%      3595.040000    5563.800000     212.416993       8.572000   
max     67474.850000  108519.280000     227.232807      14.313000   

                Size           Week           Year  
count  421570.000000  421570.000000  421570.000000  
mean   136727.915739      25.826762    2010.968591  
std     60980.583328      14.151887       0.796876  
min     34875.000000       1.000000    2010.000000  
25%     93638.000000      14.000000    2010.000000  
50%    140167.000000      26.000000    2011.000000  
75%    202505.000000      38.000000    2012.000000  
max    219622.000000      52.000000    2012.000000

In [3]:
# Number and percentage of missing values in each column and data type:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    missing_values = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    #Convert to string and add to list
    for col in data.columns: 
        dtype = str(data[col].dtype)
        types.append(dtype)
    missing_values['Types'] = types
    return(np.transpose(missing_values))

# Run function:
missing_data(walmart_data)

Store   Dept            Date Weekly_Sales IsHoliday Temperature  \
Total        0      0               0            0         0           0   
Percent      0      0               0            0         0           0   
Types    int64  int64  datetime64[ns]      float64      bool     float64   

        Fuel_Price MarkDown1 MarkDown2 MarkDown3 MarkDown4 MarkDown5      CPI  \
Total            0    270889    310322    284479    286603    270138        0   
Percent          0   64.2572    73.611   67.4808   67.9847    64.079        0   
Types      float64   float64   float64   float64   float64   float64  float64   

        Unemployment    Type   Size   Week   Year  
Total              0       0      0      0      0  
Percent            0       0      0      0      0  
Types        float64  object  int64  int64  int64

In [4]:
# Drop MarkDown columns and IsHoliday:
walmart_data = walmart_data.drop(["IsHoliday", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"], axis=1)

In [5]:
# Drop NaN values in CPI and Unemployment:
walmart_data = walmart_data.dropna()

# Display preview of dataframe:
walmart_data.head()

Store  Dept       Date  Weekly_Sales  Temperature  Fuel_Price         CPI  \
0      1     1 2010-02-05      24924.50        42.31       2.572  211.096358   
1      1     1 2010-02-12      46039.49        38.51       2.548  211.242170   
2      1     1 2010-02-19      41595.55        39.93       2.514  211.289143   
3      1     1 2010-02-26      19403.54        46.63       2.561  211.319643   
4      1     1 2010-03-05      21827.90        46.50       2.625  211.350143   

   Unemployment Type    Size  Week  Year  
0         8.106    A  151315     5  2010  
1         8.106    A  151315     6  2010  
2         8.106    A  151315     7  2010  
3         8.106    A  151315     8  2010  
4         8.106    A  151315     9  2010

In [6]:
# Change Temprature column name to Temperature F:
walmart_data=walmart_data.rename(columns={"Temperature": "Temperature F"})

# Convert temprature from Fahrenheit to Celsius:
temp_celcius=[]
for temperature in walmart_data['Temperature F']:
    celcius = (temperature - 32) * (5.0/9.0)
    temp_celcius.append(celcius)
walmart_data["Temperature C"] = temp_celcius

# Display preview:
walmart_data.head()

Store  Dept       Date  Weekly_Sales  Temperature F  Fuel_Price  \
0      1     1 2010-02-05      24924.50          42.31       2.572   
1      1     1 2010-02-12      46039.49          38.51       2.548   
2      1     1 2010-02-19      41595.55          39.93       2.514   
3      1     1 2010-02-26      19403.54          46.63       2.561   
4      1     1 2010-03-05      21827.90          46.50       2.625   

          CPI  Unemployment Type    Size  Week  Year  Temperature C  
0  211.096358         8.106    A  151315     5  2010       5.727778  
1  211.242170         8.106    A  151315     6  2010       3.616667  
2  211.289143         8.106    A  151315     7  2010       4.405556  
3  211.319643         8.106    A  151315     8  2010       8.127778  
4  211.350143         8.106    A  151315     9  2010       8.055556

In [7]:
# Rounding Column values to required decimal points:
walmart_data = walmart_data.round({'Fuel_Price': 2, 'Temperature C': 0, 'CPI': 4 })

# Display preview:
walmart_data.head()

Store  Dept       Date  Weekly_Sales  Temperature F  Fuel_Price       CPI  \
0      1     1 2010-02-05      24924.50          42.31        2.57  211.0964   
1      1     1 2010-02-12      46039.49          38.51        2.55  211.2422   
2      1     1 2010-02-19      41595.55          39.93        2.51  211.2891   
3      1     1 2010-02-26      19403.54          46.63        2.56  211.3196   
4      1     1 2010-03-05      21827.90          46.50        2.62  211.3501   

   Unemployment Type    Size  Week  Year  Temperature C  
0         8.106    A  151315     5  2010            6.0  
1         8.106    A  151315     6  2010            4.0  
2         8.106    A  151315     7  2010            4.0  
3         8.106    A  151315     8  2010            8.0  
4         8.106    A  151315     9  2010            8.0

In [8]:
# Convert Date to datetime and subtract 6 days to get Start of Week:
walmart_data['Start of Week'] = pd.to_datetime(walmart_data.Date) - timedelta(days=6)

# Export to convert date back to string and read
walmart_data.to_csv("date_to_string.csv", index_label = False)
walmart_data = pd.read_csv("date_to_string.csv")

# Display preview of dataframe:
walmart_data.head()

Store  Dept        Date  Weekly_Sales  Temperature F  Fuel_Price       CPI  \
0      1     1  2010-02-05      24924.50          42.31        2.57  211.0964   
1      1     1  2010-02-12      46039.49          38.51        2.55  211.2422   
2      1     1  2010-02-19      41595.55          39.93        2.51  211.2891   
3      1     1  2010-02-26      19403.54          46.63        2.56  211.3196   
4      1     1  2010-03-05      21827.90          46.50        2.62  211.3501   

   Unemployment Type    Size  Week  Year  Temperature C Start of Week  
0         8.106    A  151315     5  2010            6.0    2010-01-30  
1         8.106    A  151315     6  2010            4.0    2010-02-06  
2         8.106    A  151315     7  2010            4.0    2010-02-13  
3         8.106    A  151315     8  2010            8.0    2010-02-20  
4         8.106    A  151315     9  2010            8.0    2010-02-27

In [9]:
# Sort dataframe and reset index for walmart
walmart_data = walmart_data.sort_values(by =['Date']).reset_index(drop = True)

# Display preview:
walmart_data.head()

Store  Dept        Date  Weekly_Sales  Temperature F  Fuel_Price       CPI  \
0      1     1  2010-02-05      24924.50          42.31        2.57  211.0964   
1     29     5  2010-02-05      15552.08          24.36        2.79  131.5279   
2     29     6  2010-02-05       3200.22          24.36        2.79  131.5279   
3     29     7  2010-02-05      10820.05          24.36        2.79  131.5279   
4     29     8  2010-02-05      20055.64          24.36        2.79  131.5279   

   Unemployment Type    Size  Week  Year  Temperature C Start of Week  
0         8.106    A  151315     5  2010            6.0    2010-01-30  
1        10.064    B   93638     5  2010           -4.0    2010-01-30  
2        10.064    B   93638     5  2010           -4.0    2010-01-30  
3        10.064    B   93638     5  2010           -4.0    2010-01-30  
4        10.064    B   93638     5  2010           -4.0    2010-01-30

In [10]:
# Sort dataframe and reset index for walmart:
holiday_df = holiday_df.sort_values(by =['Date']).reset_index(drop = True)

# Display preview:
holiday_df.head()

Holiday  Year        Date
0    New Year's Day  2010  2010-01-01
1  Independence Day  2010  2010-07-04
2  Thanksgiving Day  2010  2010-11-25
3     Christmas Eve  2010  2010-12-24
4     Christmas Day  2010  2010-12-25

In [11]:
# List holidays we're labelling:
holiday = ['Christmas Day','Christmas Eve','Independence Day',"New Year's Day", "Thanksgiving Day"]

holiday_name = []
k = 0
# Label data with Holiday Name or No Holiday:
for i in range(len(walmart_data)):
    
    if (holiday_df['Date'][k] <= walmart_data['Date'][i]) & (holiday_df['Date'][k] >= walmart_data['Start of Week'][i]):
        holiday_name.append(holiday_df['Holiday'][k])
    
    elif (holiday_df['Date'][k] > walmart_data['Date'][i]):
        holiday_name.append("No Holiday")
    
    elif (walmart_data['Date'][i] > holiday_df['Date'][k]):
        holiday_name.append("No Holiday")
        k +=1

# Set new column as list created:
walmart_data['Holiday Name'] = holiday_name

# Convert Christmas Day and Eve to be Christmas
walmart_data['Holiday Name'] = walmart_data['Holiday Name'].replace({'Christmas Day': 'Christmas',
                                                                    'Christmas Eve': 'Christmas'})

# Display preview of dataframe:
walmart_data.head()

Store  Dept        Date  Weekly_Sales  Temperature F  Fuel_Price       CPI  \
0      1     1  2010-02-05      24924.50          42.31        2.57  211.0964   
1     29     5  2010-02-05      15552.08          24.36        2.79  131.5279   
2     29     6  2010-02-05       3200.22          24.36        2.79  131.5279   
3     29     7  2010-02-05      10820.05          24.36        2.79  131.5279   
4     29     8  2010-02-05      20055.64          24.36        2.79  131.5279   

   Unemployment Type    Size  Week  Year  Temperature C Start of Week  \
0         8.106    A  151315     5  2010            6.0    2010-01-30   
1        10.064    B   93638     5  2010           -4.0    2010-01-30   
2        10.064    B   93638     5  2010           -4.0    2010-01-30   
3        10.064    B   93638     5  2010           -4.0    2010-01-30   
4        10.064    B   93638     5  2010           -4.0    2010-01-30   

  Holiday Name  
0   No Holiday  
1   No Holiday  
2   No Holiday  
3   No Holiday  
4   No Holiday

In [12]:
# Check distribution
walmart_data.groupby('Holiday Name').count()

Store    Dept    Date  Weekly_Sales  Temperature F  \
Holiday Name                                                            
Christmas           8901    8901    8901          8901           8901   
Independence Day    8795    8795    8795          8795           8795   
New Year's Day      5900    5900    5900          5900           5900   
No Holiday        392015  392015  392015        392015         392015   
Thanksgiving Day    5959    5959    5959          5959           5959   

                  Fuel_Price     CPI  Unemployment    Type    Size    Week  \
Holiday Name                                                                 
Christmas               8901    8901          8901    8901    8901    8901   
Independence Day        8795    8795          8795    8795    8795    8795   
New Year's Day          5900    5900          5900    5900    5900    5900   
No Holiday            392015  392015        392015  392015  392015  392015   
Thanksgiving Day        5959    5959          5959    5959    5959    5959   

                    Year  Temperature C  Start of Week  
Holiday Name                                            
Christmas           8901           8901           8901  
Independence Day    8795           8795           8795  
New Year's Day      5900           5900           5900  
No Holiday        392015         392015         392015  
Thanksgiving Day    5959           5959           5959